In [1]:
from mesa import Agent, Model
from mesa.time import StagedActivation
import random
import numpy as np

random.seed(42)

In [7]:
## Parameters 
N = 10
prop_adversaries = .3
coin_toss = .8

In [15]:
class DummyAgent(Agent):
    def __init__(self, id, model):

        ## Apparently this is always needed
        super().__init__(id, model)

        

        ## Every agent has access to the model, bad design, I know, but I am lazy :( 
        self.model_state = model
        ## Unique ID, based on integer
        self.id = id

        ## This is the list of "others" ID to choose from
        self.others = list(range(0,N))
        self.others.remove(self.id)

        ## The belief vector that corresponds to the the IDs
        ## It is the belief that an agent is good
        self.belief = [1 - prop_adversaries] *  (N - 1)

        ## The belief vector that corresponds to the the IDs
        self.votes = []

    ## Should be fixed for every agent
    def select_from_belief(self):

        ## For conversion to a valid probability distribution
        normalizing_constant = sum(self.belief)
        
        ## If the agent does not trust anyone, then
        if normalizing_constant == 0:
            ## No one was chosen
            self.model_state.chose = False
            return -1
        
        else:

            selection_probability = [b / normalizing_constant for b in self.belief]
            # Using random.choices() for a single selection
            selected_item = random.choices(self.others, weights=selection_probability, k=1)[0]

            return selected_item
    
    def choose(self):

        # Implementation of choose action
        if self.model_state.whose_turn == self.id:

            choice = self.select_from_belief()
            self.model_state.chosen.append(choice)
            
        else:
            pass
    
    def vote(self):
        
        ## if someone is chosen
        if self.model_state.chose:

            chooser = self.model_state.whose_turn
            chosen = self.model_state.chosen[-1]

            belief_chooser, belief_chosen = None, None

            ## If you are the chooser
            if chooser == self.id:
                belief_chooser = 1
            ## If you are the chosen
            elif chosen == self.id:
                belief_chosen = 1
            
            ## What to do if you are neither
            ## Then look at my current belief
            if belief_chooser is None:
                belief_chooser = self.belief[self.others.index(chooser)]
            if belief_chosen is None:
                belief_chosen = self.belief[self.others.index(chosen)]

            ## What is your vote?
            if belief_chooser + belief_chosen > 1:
                ## TODO
                current_vote = random.randint(0, 1)
            else:
                ## TODO
                current_vote = 0

        ## if nobody was chosen
        else: 

            current_vote = -1

        ## track my current vote
        self.votes.append(current_vote)
        ## update the model state
        self.model_state.votes_round.append(current_vote)
        
    
    def act(self):
        
        ## what if no one was chosen, then no action
        if self.model_state.chose is not True:
            pass

        ## vote outcome must be a strict majority
        vote_outcome = sum(self.model_state.votes_round)/len(self.model_state.votes_round)
        if vote_outcome <= 0.5:
            ## TODO, what happens if fail
            pass

        ## action for chooser 
        if self.id == self.model_state.whose_turn:
            action = random.randint(0, 1)
            self.model_state.action_round[0] = action
        ## action for chosen
        elif self.id == self.model_state.chosen[-1]:
            action = random.randint(0, 1)
            self.model_state.action_round[1] = action

        else:
            pass

    def update(self):

        pass


class MyModel(Model):
    def __init__(self, N=N):
        
        super().__init__()

        assert N > 0, "N must be greater than 0."
        self.num_agents = N

        ## Staged activation to determine which round
        ## And whose turn to be the chooser
        self.schedule = StagedActivation(self, stage_list=["choose", "vote", "act", "update"])
        self.round = 0 
        self.whose_turn = self.round % N

        ## if anyone was chosen
        self.chose = True
        ## who is the chosen, and -1 flags is that no one was
        self.chosen = []

        ## voting record of across the rounds
        self.voting_record = []
        ## voting record of the current round
        self.votes_round = []

        ## Action
        self.action_record = []
        self.action_round = [-1,-1]

        ## Outcome 
        self.outcome = []
        
        # Change this up a little
        for i in range(self.num_agents):  # Example: creating 10 agents
            agent = DummyAgent(i, self)
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()
        self.round += 1

        ## determine which team won in given round
        ## 1 = Good team wins, 0 = Bad team wins
        if self.action_round[0] == 1 and self.action_round[1] == 1:
            self.outcome.append(1)
        else:
            self.outcome.append(0)

        ## Reset Parameters
        self.whose_turn = self.round % N
        self.chose = True
        self.voting_record.append(self.votes_round)
        self.votes_round = []
        self.action_round = [-1,-1]


In [16]:
model = MyModel(N)

for i in range(10):
    model.step()


In [18]:
model.outcome

[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [11]:
# Example lists
items = ['apple', 'banana', 'cherry']
probabilities = [0.1, 0.6, 0.3]

# Using random.choices() for a single selection
selected_item = random.choices(items, weights=probabilities, k=1)[0]
selected_item

'apple'

In [6]:
selected_item

'cherry'

In [18]:
[0.1, 0.6, 0.3] / sum([0.1, 0.6, 0.3])

TypeError: unsupported operand type(s) for /: 'list' and 'float'